## Setup


## Required Libraries

For this lab, we will be using the following Python libraries:

* [`crewai`](https://pypi.org/project/crewai/) – The core framework for building collaborative AI workflows using agents, tasks, and process management.
* [`crewai-tools`](https://pypi.org/project/crewai-tools/) – A set of prebuilt tools (like web search, file I/O, and APIs) that can be used by CrewAI agents.
* [`langchain`](https://www.langchain.com/) – Provides core utilities for working with LLMs, prompts, tools, and memory management (used under the hood by CrewAI).
* [`langchain-community`](https://pypi.org/project/langchain-community/) – Offers integration with open-source and third-party tools used in the broader LangChain ecosystem.


### Installing Required Libraries

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [4]:
%pip install langchain==0.3.20 | tail -n 1 
%pip install crewai==0.80.0 | tail -n 1
%pip install langchain-community==0.3.19 | tail -n 1 
%pip install crewai-tools==0.38.0 | tail -n 1
%pip install databricks-sdk==0.57.0| tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


----


## **What is CrewAI?**  

CrewAI is a **cutting-edge framework** that empowers us to create and manage teams of **autonomous AI agents** designed to collaborate on complex tasks. Think of it as our ultimate toolkit for assembling a team of virtual experts, where each member plays a **specific role**, uses **unique tools**, and works toward **clear goals**. These agents aren’t just working in isolation; they collaborate, communicate, and solve problems as a synchronized team, enabling us to achieve more than ever before.   

### **Why CrewAI?**  
Imagine you’re leading a project. You need specialists—each with unique expertise—who can work together to achieve a common goal. CrewAI replicates this dynamic in the world of AI by:  
- Assigning **roles** to agents based on their purpose (e.g., a planner, an executor, or a coordinator).  
- Equipping them with **tools** to perform their tasks efficiently.  
- Directing them with **goals** to ensure their efforts align with the broader mission.  

This collaborative framework ensures that your AI agents can tackle challenges that are too big or too complex for a single agent to handle. Whether it's **automation**, **decision-making**, or **simulating real-world scenarios**, CrewAI empowers you to orchestrate your AI teams like never before.  

### **How CrewAI Works**
At its core, CrewAI provides us with a high-level framework to build “crews”—groups of role-playing agents that interact and collaborate to achieve shared objectives. Each agent is:  
- **Assigned a Role:** Just like in a real team, every agent has a specialized function, whether it’s planning, executing, or coordinating tasks.  
- **Equipped with Tools:** Agents are provided with the resources they need to perform their roles effectively.  
- **Directed by Goals:** Clear objectives ensure that every agent’s efforts align with the crew’s mission.  


## Setting Up SerperDevTool

**What is Serper?**  
Serper is a real-time Google Search API that allows AI agents to access up-to-date web information—effectively connecting your workflow to the latest content on the internet.

**Why are we using Serper in our workflow?**  
Our research agent needs current, reliable information to uncover trends, breakthroughs, and insights on evolving topics like generative AI, quantum computing, or sustainability. Without web access, the agent would be limited to static, pre-trained knowledge and unable to reflect the latest developments.

To use the `SerperDevTool`, it requires an **API key**. This key grants access to the web search service and allows our agents to fetch real-time data during execution.

> You will need to obtain your API Key from [serper.dev](https://serper.dev).  
> - Sign up or log in with your email  
> - Navigate to the **Dashboard**  
> - Click on **API Keys**  
> - Copy the key and replace `API_KEY` in your code with the value provided

To learn more about the `SerperDevTool` and its capabilities, visit the [official documentation](https://serper.dev/).


Enter  API key 


In [7]:
import os 
os.environ['SERPER_API_KEY'] = "your_serper_api_key"

Import ```SerperDevTool``` from ```crewai_tools```. 


In [8]:
%%capture

from crewai_tools import SerperDevTool

Initialize the SerperDev search tool  object (requires an API key)


In [9]:
search_tool=SerperDevTool()
print(type(search_tool))

<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevTool'>


Run a search query 


In [10]:
search_query = "Latest Breakthroughs in machine learning"
search_results =search_tool.run(query=search_query )

# Print the results
print(f"Search Results for '{search_results}':\n")

Using Tool: Search the internet with Serper
Search Results for '{'searchParameters': {'q': 'Top IT company in Bangladesh', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Top 20 Best Software Companies in Bangladesh (2025) | Vivasoft Ltd.', 'link': 'https://vivasoftltd.com/best-software-companies-in-bangladesh/', 'snippet': 'Vivasoft Limited is the best software company in Bangladesh, mainly providing custom software development solutions.', 'position': 1}, {'title': 'Top 10 Tech Companies in Bangladesh | StartUs Insights', 'link': 'https://www.startus-insights.com/innovators-guide/tech-companies-in-bangladesh/', 'snippet': '1. Pulse Tech · 2. WeGro · 3. RyoGas · 4. Nexis · 5. Tiger New Energy · 6. Restora POS · 7. Shiyo Tech · 8. Reinforz AI.', 'position': 2}, {'title': 'Discover the Best Software Development Company in Bangladesh', 'link': 'https://www.reddit.com/r/TopSoftwareDeveloper/comments/1j21g9c/discover_the_best_software_development_company_in/', 'sni

The ````search_results```` dictionary has a lot of info, so here is an overview of what each key contains:


- **searchParameters**: Query metadata (term, engine, result count)
- **organic**: Search results (title, link, snippet, position)
- **peopleAlsoAsk**: Related questions with answers
- **relatedSearches**: Alternative search queries
- **credits**: API usage tracking


In [11]:
print("keys of search_results", search_results.keys())

keys of search_results dict_keys(['searchParameters', 'organic', 'peopleAlsoAsk', 'relatedSearches', 'credits'])


## Setting up our LLM

Next, we need to set up our **LLM (Large Language Model)**—this can be **any model** based on our needs. Here, we are going to use **Meta Llama 3.3 70b instruct**. The choice of model depends on factors such as **accuracy, speed, and recipe understanding** for our meal planning tasks.


In [13]:
from crewai import LLM

llm = LLM(
        model="watsonx/meta-llama/llama-3-3-70b-instruct",
        base_url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        max_tokens=2000,
)

## **Agents in CrewAI**  

In CrewAI, **agents** are the foundational units of any multi-agent system. Each agent is designed to perform a specific role, solve tasks autonomously, and collaborate seamlessly with other agents. They’re more than mere programs—they are your specialized team members in an AI-powered ecosystem.  

---




A CrewAI agent isn’t just a block of code; it’s a thoughtfully designed entity with the following parameters:  

1. **Role**  
   An agent’s role defines its purpose in the system. Roles are as diverse as your project needs, such as a **"Data Researcher"** hunting for insights or a **"Reporting Analyst"** preparing comprehensive summaries.  

2. **Goal**  
   Each agent operates with a defined goal—a guiding star that shapes its decisions and actions. For instance, an agent with the goal to **“Uncover cutting-edge developments in AI”** will consistently align its behavior to fulfill this objective.  

3. **Backstory**  
   An agent’s backstory is like its resume, providing context or personality that influences how it behaves and interacts. For example, a seasoned **“Senior Data Researcher”** with years of experience might approach tasks differently from a **“Junior Analyst”** just starting out. This feature adds depth and relatability to agent interactions, making them more dynamic and tailored.  


4. **Tools**  
   Just like any professional needs the right tools to excel, agents in CrewAI are equipped with specialized tools to boost their performance. Whether it’s a **web search utility** for gathering information, a **data analysis engine** for crunching numbers, or an **API connector** to integrate external services, tools expand an agent’s capabilities. The right tool can help an agent complete its tasks more efficiently and effectively, enabling it to work smarter, not harder.  

5. **Configuration**  
   Agents in CrewAI are configured using simple YAML files, offering a modular, readable, and scalable approach to defining their attributes. This makes setting up agents intuitive, even for large systems ( in this tutroal we will not use a YML files 




####  **Defining an Agent Directly as a Python Object**
For more flexibility or when working in a programmatic environment, you can define agents directly in your code. This approach allows you to quickly integrate dynamic parameters and logic into the agent’s setup.

In this section, we're defining the research agent which will gather and analyze information from the web. This "Senior Research Analyst" uses the SerperDevTool to search for relevant content, working independently without delegation. The agent serves as the first step in our workflow, collecting the raw data that other agents will later refine and present.

Example of defining an agent in **Python**:



In [14]:
from crewai import Agent

research_agent = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge information and insights on any subject with comprehensive analysis',
  backstory="""You are an expert researcher with extensive experience in gathering, analyzing, and synthesizing information across multiple domains. 
  Your analytical skills allow you to quickly identify key trends, separate fact from opinion, and produce insightful reports on any topic. 
  You excel at finding reliable sources and extracting valuable information efficiently.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[SerperDevTool()]
)


In this Python example, an agent is created with the same role, goal, backstory, and tools as the YAML example. However, this method allows you to easily pass in dynamic variables and parameters at runtime, making it ideal for scenarios where the agent configuration needs to change dynamically.


In [15]:
research_agent

Agent(role=Senior Research Analyst, goal=Uncover cutting-edge information and insights on any subject with comprehensive analysis, backstory=You are an expert researcher with extensive experience in gathering, analyzing, and synthesizing information across multiple domains. 
  Your analytical skills allow you to quickly identify key trends, separate fact from opinion, and produce insightful reports on any topic. 
  You excel at finding reliable sources and extracting valuable information efficiently.)

In CrewAI, we use multiple specialized agents to complete complex tasks through collaboration. In our research-report example:

1. We created a **Researcher Agent** that gathers information
2. Now we will create a **Writer Agent** that takes the output from our Researcher Agent
3. The Writer transforms research findings into well-structured content for the target audience

Let's create the writer agent with the following parameters:

* **role**: 'Tech Content Strategist' - Job function within the workflow
* **goal**: 'Craft well-structured and engaging content based on research findings' - The agent's specific objective
* **backstory**: Background that shapes the agent's approach and style
* **verbose**: True - Controls logging detail level
* **allow_delegation**: True - Enables task assignment to other agents


In [16]:
# Define your agents with roles and goals
# Define the Writer Agent
writer_agent = Agent(
  role='Tech Content Strategist',
  goal='Craft well-structured and engaging content based on research findings',
  backstory="""You are a skilled content strategist known for translating 
  complex topics into clear and compelling narratives. Your writing makes 
  information accessible and engaging for a wide audience.""",
  verbose=True,
  llm = llm,
  allow_delegation=True
)

In [17]:
writer_agent 

Agent(role=Tech Content Strategist, goal=Craft well-structured and engaging content based on research findings, backstory=You are a skilled content strategist known for translating 
  complex topics into clear and compelling narratives. Your writing makes 
  information accessible and engaging for a wide audience.)

## **Tasks in CrewAI**
Tasks are like to-do items for our AI agents. Each task has specific instructions, details, and tools for the agent to follow and complete the job.

For example:
- A task could ask an agent to "research the latest AI trends."
- Another task could ask a different agent to "write a detailed report based on the research."



Here is an outline of the porcess:

1. **Define agents** with their roles, goals, and tools
2. **Create tasks** and assign them to specific agents
3. **Combine agents and tasks** into a Crew with an execution process


#### **How Tasks Work**  
There are two ways tasks can run:  

1. **Sequential**: Tasks are executed one after the other, like following a recipe step-by-step. Each task waits for the previous one to finish.  
2. **Hierarchical**: Tasks are assigned based on agent skills or roles, and multiple tasks can run in parallel if they don’t depend on each other.  




#### **What Can a Task Include?**
Each task has these details:
- **Description**: What needs to be done.
- **Expected Output**: What the result should look like.
- **Agent**: Who’s responsible for the task.
- **Tools**: The tools the agent can use for this task.
- **Context**: Outputs from other tasks that help this task.
- **Async Execution**: Whether the task runs in the background or not.
- **Output Format**: Whether the results are plain text, JSON, or a structured model.


Here's how we set up a Crew (our team of agents) and tasks in code `research_task` and `writer_task`. 

In this step, we define a Task for the Researcher Agent. This task will involve gathering and analyzing key insights on any topic specified through the `{topic}` parameter. The agent will use the SerperDevTool to uncover major trends, identify new technologies, and evaluate their effects on the industry. This flexible approach allows us to research different subjects by simply changing the input parameter when kicking off the crew.


In [19]:
from crewai import Task

research_task = Task(
  description="Analyze the major {topic}, identifying key trends and technologies. Provide a detailed report on their potential impact.",
  agent=research_agent,
  expected_output="A detailed report on {topic}, including trends, emerging technologies, and their impact."
)

Now, we will define the task for the Writer Agent, who will take the research findings and transform them into a well-structured article. The Writer Agent will ensure the content is engaging, informative, and easy to understand, making complex topics more accessible.


In [20]:
# Create a task for the Writer Agent
writer_task = Task(
  description="Create an engaging blog post based on the research findings about {topic}. Tailor the content for a tech-savvy audience, ensuring clarity and interest.",
  agent=writer_agent,
  expected_output="A 4-paragraph blog post on {topic}, written clearly and engagingly for tech enthusiasts."
)

## CrewAI Workflow

The  `Crew` object, which is the central orchestration mechanism in CrewAI. This crew brings together our specialized agents and their assigned tasks into a cohesive workflow.

The `Crew` constructor takes several important parameters:
- `agents`: A list of the AI agents that will be part of this crew ```research_agent``` abd  ```writer_agent```
- `tasks`: A list of specific tasks these agents will perform ```research_task``` and ```writer_task```
- `process`: Defines how tasks will be executed - in this case `Process.sequential means tasks will run one after another in the specified order (research first, then writing)
- `verbose`: When set to `True`, this enables detailed logging, making it easier to follow the crew's execution and troubleshoot any issues

Once configured, you can start the entire workflow with a single command: `crew.kickoff()`, which will execute the tasks in sequence and return the final results.


In [22]:
from crewai import Crew, Process

crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writer_task],
    process=Process.sequential,
    verbose=True 
)

The method ```kickoff()``` sets everything rolling - it starts all your agents working on their tasks and returns the results when they're done. By using ```inputs={"topic": "quantum computing breakthroughs of 2024"}```, we can specify exactly what subject our agents should research, making our system flexible enough to analyze any topic without changing the task definitions.


In [23]:
result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d4a6be2-bed8-4b1a-99b1-b61315a8040a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide   │
│  a detailed report on their potential impact.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: To analyze the major latest generative AI breakthroughs, I need to search for the most       │
│  recent and relevant information on this topic. This will involve looking into current research, developments,  │
│  and innovations in the field of generative AI.                                                                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest generative AI breakthroughs\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6056e072-1ba3-4f47-a269-69de6d6d5acd                                                                     │
│  Agent: Tech Content Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0d4a6be2-bed8-4b1a-99b1-b61315a8040a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The latest generative AI breakthroughs are transforming industries and revolutionizing the way   │
│  we live and work. One of the key trends in generative AI is hyper-personalization, which enables machines to   │
│  create personalized content recommendations, adaptive user interfaces, and individualized learning materials.  │
│  Another trend is conversational AI, which allows humans to interact with machines in a more natural and        │
│  intuitive way. Multi-modal AI is also gaining traction, enabling machines to understand and generate multiple  │
│  forms of media, such as text, images, and videos.                                                              │
│                                                                                                                 │
│  Generative AI has numerous applications and impact across various industries. In the creative industries,      │
│  generative AI can be used to generate new ideas, automate repetitive tasks, and enhance human creativity. In   │
│  healthcare, generative AI can be used to analyze medical images, diagnose diseases, and develop personalized   │
│  treatment plans. In finance, generative AI can be used to detect fraud, predict market trends, and optimize    │
│  investment portfolios. However, the growing importance of AI ethics and regulation cannot be overlooked. As    │
│  generative AI becomes more pervasive, it is essential to ensure that it is developed and used in a             │
│  responsible and transparent manner.                                                                            │
│                                                                                                                 │
│  The potential of generative AI to transform a wide range of industries and areas is vast. According to a       │
│  report by McKinsey, generative AI could add trillions of dollars in value to the global economy. However, the  │
│  development and deployment of generative AI also come with environmental consequences, such as increased       │
│  electricity demand and water usage. Therefore, it is crucial to develop sustainable and                        │
│  environmentally-friendly generative AI solutions. Furthermore, the use of generative AI raises important       │
│  questions about the future of work, as it has the potential to automate jobs traditionally considered immune   │
│  to automation.                                                                                                 │
│                                                                                                                 │
│  In conclusion, the latest generative AI breakthroughs are revolutionizing industries and transforming the way  │
│  we live and work. As generative AI continues to evolve and improve, it is essential to ensure that it is       │
│  developed and used in a responsible and transparent manner, with consideration for its potential impact on     │
│  the environment and society. By harnessing the power of generative AI, we can unlock new opportunities for     │
│  growth, innovation, and progress, and create a better future for all. With the ability to generate new         │
│  content, automate repetitive tasks, and enhance human

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

 y


The result is a ```crew_output``` 


In [24]:
type(result)

crewai.crews.crew_output.CrewOutput

In [25]:
result

CrewOutput(raw='The latest generative AI breakthroughs are transforming industries and revolutionizing the way we live and work. One of the key trends in generative AI is hyper-personalization, which enables machines to create personalized content recommendations, adaptive user interfaces, and individualized learning materials. Another trend is conversational AI, which allows humans to interact with machines in a more natural and intuitive way. Multi-modal AI is also gaining traction, enabling machines to understand and generate multiple forms of media, such as text, images, and videos.\n\nGenerative AI has numerous applications and impact across various industries. In the creative industries, generative AI can be used to generate new ideas, automate repetitive tasks, and enhance human creativity. In healthcare, generative AI can be used to analyze medical images, diagnose diseases, and develop personalized treatment plans. In finance, generative AI can be used to detect fraud, predict

The `result.raw` output text contains the final content produced by our last agent in the workflow. We can easily access this text to see the complete results:


In [26]:
final_output = result.raw
print("Final output:", final_output)

Final output: The latest generative AI breakthroughs are transforming industries and revolutionizing the way we live and work. One of the key trends in generative AI is hyper-personalization, which enables machines to create personalized content recommendations, adaptive user interfaces, and individualized learning materials. Another trend is conversational AI, which allows humans to interact with machines in a more natural and intuitive way. Multi-modal AI is also gaining traction, enabling machines to understand and generate multiple forms of media, such as text, images, and videos.

Generative AI has numerous applications and impact across various industries. In the creative industries, generative AI can be used to generate new ideas, automate repetitive tasks, and enhance human creativity. In healthcare, generative AI can be used to analyze medical images, diagnose diseases, and develop personalized treatment plans. In finance, generative AI can be used to detect fraud, predict mar

The `tasks_output` list gives us access to outputs from each task in the order they were executed:


In [29]:
tasks_outputs = result.tasks_output

We see the output of the research task object. This lets us access both the task description and the content the agent produced:


In [30]:
print("Task Description", tasks_outputs[0].description)
print("Output of research task ",tasks_outputs[0])

Task Description Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide a detailed report on their potential impact.
Output of research task  The latest generative AI breakthroughs have been making waves in the technology and scientific communities. Recent advancements include the development of a generative AI system named FlowER (Flow matching for Electron Redistribution) that predicts chemical reactions while strictly enforcing conservation of mass and electrons. This breakthrough has the potential to revolutionize the field of chemistry and materials science.

Other notable trends and technologies in generative AI include:

1. AI for Creativity: Generative AI is being used to create art, music, and other forms of creative content.
2. GenAI for Hyper-Personalization: Generative AI is being used to create personalized content and recommendations for individuals.
3. Conversational AI: Generative AI is being used to create chatbots and ot

We also have the description and output for the writer task using the raw property:


In [31]:
print("Writer task description:", tasks_outputs[1].description)
print(" \nOutput of writer task:", tasks_outputs[1].raw)

Writer task description: Create an engaging blog post based on the research findings about Latest Generative AI breakthroughs. Tailor the content for a tech-savvy audience, ensuring clarity and interest.
 
Output of writer task: The latest generative AI breakthroughs are transforming industries and revolutionizing the way we live and work. One of the key trends in generative AI is hyper-personalization, which enables machines to create personalized content recommendations, adaptive user interfaces, and individualized learning materials. Another trend is conversational AI, which allows humans to interact with machines in a more natural and intuitive way. Multi-modal AI is also gaining traction, enabling machines to understand and generate multiple forms of media, such as text, images, and videos.

Generative AI has numerous applications and impact across various industries. In the creative industries, generative AI can be used to generate new ideas, automate repetitive tasks, and enhanc


In addition to the task output, we can access the agent that performed each task:


In [32]:
print("We can get the agent for researcher task:  ",tasks_outputs[0].agent)
print("We can get the agent for the writer task: ",tasks_outputs[1].agent)

We can get the agent for researcher task:   Senior Research Analyst
We can get the agent for the writer task:  Tech Content Strategist


---
After your agents complete their tasks, CrewAI provides detailed performance metrics that help you monitor resource usage and optimize your multi-agent systems. Token usage analytics are particularly important as they directly impact operational costs and system efficiency.


In [33]:
token_count = result.token_usage.total_tokens
prompt_tokens = result.token_usage.prompt_tokens
completion_tokens = result.token_usage.completion_tokens

print(f"Total tokens used: {token_count}")
print(f"Prompt tokens: {prompt_tokens} (used for instructions to the model)")
print(f"Completion tokens: {completion_tokens} (generated in response)")

Total tokens used: 24066
Prompt tokens: 20631 (used for instructions to the model)
Completion tokens: 3435 (generated in response)


In [34]:
social_agent = Agent(
    role="Social Media Strategist",
    goal="Curate engaging summaries and social posts that promote fact-checked content and drive user engagement.",
    backstory="""A creative social media expert skilled in turning fact-checking reports into engaging short-form 
    content for Twitter and LinkedIn.""",
    verbose=True,
    llm = llm,
    allow_delegation=False
)

In [35]:
social_task = Task(
    description=(
        "Summarize the blog post about {topic} into 2–3 engaging social media posts "
        "suitable for platforms like LinkedIn or Twitter. Make sure the tone is informative, "
        "professional, and encourages further reading."
    ),
    agent=social_agent,
    expected_output="A series of 2–3 well-written social posts highlighting the key insights from the blog content."
)

In [ ]:
crew = Crew(
    agents=[research_agent, writer_agent, social_agent],
    tasks=[research_task, writer_task, social_task],
    process=Process.sequential,  # Tasks will be executed one after another
    verbose=True
) 

result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2d9b3372-7ac2-4633-8ece-ea9c813e47e8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide   │
│  a detailed report on their potential impact.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: To analyze the major latest generative AI breakthroughs and identify key trends and          │
│  technologies, I need to start by searching for the most recent and relevant information on this topic. This    │
│  will involve looking into various sources that discuss advancements, innovations, and applications in the      │
│  field of generative AI.                                                                                        │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest generative AI breakthroughs\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: To research the latest developments in generative AI, I should start by searching the        │
│  internet for the most recent and relevant information on this topic, focusing on key trends, emerging          │
│  technologies, and future directions in the field.                                                              │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest developments in generative AI\"}"                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'latest developments in generative AI', 'type': 'search', 'num': 10, 'engine':      │
│  'google'}, 'organic': [{'title': 'What does the future hold for generative AI? | MIT News', 'link':            │
│  'https://news.mit.edu/2025/what-does-future-hold-generative-ai-0919', 'snippet': 'Several MIT faculty members  │
│  also spoke about their latest research projects, including the use of AI to reduce noise in ecological image   │
│  data, ...', 'position': 1}, {'title': 'Generative AI news and analysis - TechCrunch', 'link':                  │
│  'https://techcrunch.com/tag/generative-ai/', 'snippet': 'TechCrunch covers the latest news and analysis in     │
│  Generative AI (GenAI). Read the latest on startups and founders building artificial intelligence tools.',      │
│  'position': 2}, {'title': 'Generative AI', 'link': 'https://generativeai.pub/', 'snippet': 'Stay updated with  │
│  the latest news, research, and developments in the world of generative AI. We cover everything from AI model   │
│  updates, ...', 'position': 3}, {'title': 'The 2025 AI Index Report | Stanford HAI', 'link':                    │
│  'https://hai.stanford.edu/ai-index/2025-ai-index-report', 'snippet': 'In 2023, researchers introduced new      │
│  benchmarks—MMMU, GPQA, and SWE-bench—to test the limits of advanced AI systems. Just a year later,             │
│  performance sharply ...', 'position': 4}, {'title': 'The Latest AI News and AI Breakthroughs that Matter       │
│  Most: 2025', 'link': 'https://www.crescendo.ai/news/latest-ai-news-and-updates', 'snippet': 'Summary: MIT      │
│  researchers have developed a generative AI system named FlowER (Flow matching for Electron Redistribution)     │
│  that predicts chemical reactions while ...', 'position': 5}, {'title': "6 AI trends you'll see more of in      │
│  2025 - Microsoft Source", 'link':                                                                              │
│  'https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/', 'snippet': 'In the     │
│  last year alone, generative AI usage jumped from 55% to 75% among business leaders and AI decisionmakers. New  │
│  AI tools will bring ...', 'position': 6}, {'title': "Generative AI: Here's...                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()